# Proposal for Government Zonal Hospitals - Mumbai

## Introduction/Business Problem

Mumbai in India, stands at 6th place in the list of “Cities by population density” of the world. 
Traffic is also pretty bad there. Many a times ambulances are stuck in traffic and reaching to the nearest Government hospital becomes very difficult.
Keeping all this in mind, it would be better to recognise at least one hospital in a zone, where treatment can be given at subsidised cost.

To achieve this, 
1. Divide the city into Zones.
2. Consider Postal codes for City Division.
3. In additional to that, cluster the Postal codes into ‘K’ clusters.
4. Check if there is at least one hospital in every zone.
5. List out all the hospitals in the zone. This is to give options to pick the hospital for government affiliation.
6. If there is no hospital in the zone, check if there is at least one hospital in the clusters( to which this zone belongs to).
7. List at least one hospital, from the list of hospitals in the zone, to tag to the zones which does not have hospitals otherwise.
8. If there is not hospital even in the cluster, then that would indicate that there is a need for the hospital in that area. Suggest government to build at least one hospital in that zone/cluster.
9. Finally, there will be a list of areas and for every area there will be either a list of hospitals or at least one hospital that can be used for government association.

## Data section

To get the postal codes of Mumbai, referred to https://worldpostalcode.com/india/maharashtra/mumbai/.

By web-Scraping extract area, pincode.

Using 'geolocator' find the latlong for pincode.

With this latlong, use foursquare API to find the all the hospitals in the area of the Pincode.

Cluster all the areas(using pincodes) into an appropriate number of clusters by using KMeans.

Now, use foursquare API to find all the hospitals in every pincode using latlong

Check if any area does not have a hospital, in which case check if the cluster(to which this area belongs), has a hospital

Pick any hospital from the cluster hospital list and suggest to this area.

Finally, there will be either a list of hospital suggestions(in case of hospital in the zonal area) or atleast one hospital suggestion(in case of hospital in the cluster area)